In [54]:
## columns to drop
# How can we use transaction number and date as categorical values?
# drop transaction number
# should we filter out transaction sub type column?
# is free hold? turn into dummy variables.
# filtering out usage as residential - elimiates 4263 data points.
# should we filter out land from the property type column?
# property sub type? it lists things like clinic
#drop property sub type column, keep property type
#dropping transaction size because there are blank values. keeping property size.
#how should we filter out the rooms column? should we create an "other" category out of all the subcategories that aren't listed: gym, hotel, shop
# use KNN method to predict room values 
# use get dummies for parking column
# drop nearest metro, mall, and landmark categories.

In [33]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


In [34]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("resources/trimmed.csv")
df = pd.read_csv(file_path)
# Review the DataFrame
df.head()

,Transaction Date,Transaction Type,Transaction sub type,Registration type,Is Free Hold?,Usage,Area,Property Type,Amount,Property Size (sq.m),Room(s)
0,2023-01-02 06:56:17,Sales,Sale,Ready,Free Hold,Residential,ARJAN,Unit,1056207.41,94.05,1 B/R
1,2023-01-02 07:19:57,Sales,Sale,Ready,Non Free Hold,Residential,Al Wasl,Land,5000000.00,1021.93,NaN
2,2023-01-02 07:25:49,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,BUSINESS BAY,Unit,2631000.00,105.75,2 B/R
3,2023-01-02 07:30:03,Sales,Delayed Sell,Ready,Free Hold,Residential,Al Hebiah Fifth,Land,2039000.00,143.99,NaN
4,2023-01-02 07:35:30,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,SOBHA HEARTLAND,Unit,3126741.00,147.08,3 B/R


In [ ]:
# Drop null values in rooms column
df.dropna(subset=['Room(s)'])

room_counts = df['Room(s)'].value_counts()
room_counts

Room(s)
1 B/R          23521
2 B/R          16128
Studio         11222
3 B/R           9423
4 B/R           3391
Office          1751
Shop             705
5 B/R            329
PENTHOUSE         63
6 B/R             33
Single Room        8
GYM                3
Hotel              2
7 B/R              2
9 B/R              1
Name: count, dtype: int64

In [ ]:
# Drop transaction date column
new_df = df.drop(columns=['Transaction Date'])
new_df

,Transaction Type,Transaction sub type,Registration type,Is Free Hold?,Usage,Area,Property Type,Amount,Property Size (sq.m),Room(s)
0,Sales,Sale,Ready,Free Hold,Residential,ARJAN,Unit,1056207.41,94.05,1 B/R
1,Sales,Sale,Ready,Non Free Hold,Residential,Al Wasl,Land,5000000.00,1021.93,NaN
2,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,BUSINESS BAY,Unit,2631000.00,105.75,2 B/R
3,Sales,Delayed Sell,Ready,Free Hold,Residential,Al Hebiah Fifth,Land,2039000.00,143.99,NaN
4,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,SOBHA HEARTLAND,Unit,3126741.00,147.08,3 B/R
...,...,...,...,...,...,...,...,...,...,...
81596,Sales,Sale,Ready,Free Hold,Residential,BURJ KHALIFA,Unit,1790000.00,75.07,1 B/R
81597,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,DUBAI SPORTS CITY,Unit,229850.00,31.64,Studio
81598,Sales,Sell - Pre registration,Off-Plan,Free Hold,Residential,AL FURJAN,Unit,2250000.00,272.59,3 B/R
81599,Mortgage,Mortgage Registration,Ready,Free Hold,Residential,DUBAI CREEK HARBOUR,Unit,1242438.00,106.71,2 B/R


In [ ]:
# Categorize features depending on their datatype.
obj = (new_df.dtypes == 'object')
object_cols = list(obj[obj].index)
print("Categorical variables:",len(object_cols))
 
int_ = (new_df.dtypes == 'int')
num_cols = list(int_[int_].index)
print("Integer variables:",len(num_cols))
 
fl = (new_df.dtypes == 'float')
fl_cols = list(fl[fl].index)
print("Float variables:",len(fl_cols))
new_df

Categorical variables: 0
Integer variables: 0
Float variables: 2


,Amount,Property Size (sq.m),Transaction Type_Gifts,Transaction Type_Mortgage,Transaction Type_Sales,Transaction sub type_Adding Land By Sell,Transaction sub type_Delayed Development,Transaction sub type_Delayed Mortgage,Transaction sub type_Delayed Sell,Transaction sub type_Delayed Sell Development,...,Room(s)_6 B/R,Room(s)_7 B/R,Room(s)_9 B/R,Room(s)_GYM,Room(s)_Hotel,Room(s)_Office,Room(s)_PENTHOUSE,Room(s)_Shop,Room(s)_Single Room,Room(s)_Studio
0,1056207.41,94.05,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5000000.00,1021.93,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2631000.00,105.75,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2039000.00,143.99,False,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,3126741.00,147.08,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81596,1790000.00,75.07,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
81597,229850.00,31.64,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
81598,2250000.00,272.59,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
81599,1242438.00,106.71,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [56]:
# Heatmap using seaborn.
#plt.figure(figsize=(12, 6))
#sns.heatmap(dummies_df.corr(),
 #           cmap = 'BrBG',
  #          fmt = '.2f',
   #         linewidths = 2,
    #        annot = True)

In [57]:
# Analyze categorical features.

#unique_values = []
#for col in object_cols:
#  unique_values.append(dummies_df[col].unique().size)
#plt.figure(figsize=(10,6))
#plt.title('No. Unique values of Categorical Features')
#plt.xticks(rotation=90)
#sns.barplot(x=object_cols,y=unique_values)

In [ ]:
df.head()

In [ ]:
# Replacing amount empty values with their mean values to make the data distribution symmetric.
df['Amount'] = df['Amount'].fillna(
  df['Amount'].mean())

new_dataset = df.dropna()
new_dataset.isnull().sum()


In [ ]:
from sklearn.preprocessing import OneHotEncoder
 
s = (new_dataset.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)
print('No. of. categorical features: ',
      len(object_cols))

In [ ]:
# One hot encode whole categorical list.
OH_encoder = OneHotEncoder(sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(new_dataset[object_cols]))
OH_cols.index = new_dataset.index
OH_cols.columns = OH_encoder.get_feature_names()
df_final = new_dataset.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

In [58]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
 
X = dummies_df.drop(['Amount'], axis=1)
Y = dummies_df['Amount']
 
# Split the training set into
# training and validation set
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X, Y, train_size=0.8, test_size=0.2, random_state=0)
model_SVR = svm.SVR()
model_SVR.fit(X_train,Y_train)
Y_pred = model_SVR.predict(X_valid)
 
print(mean_absolute_percentage_error(Y_valid, Y_pred))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
 
model_RFR = RandomForestRegressor(n_estimators=10)
model_RFR.fit(X_train, Y_train)
Y_pred = model_RFR.predict(X_valid)
 
mean_absolute_percentage_error(Y_valid, Y_pred)

In [ ]:
from sklearn.linear_model import LinearRegression
 
model_LR = LinearRegression()
model_LR.fit(X_train, Y_train)
Y_pred = model_LR.predict(X_valid)
 
print(mean_absolute_percentage_error(Y_valid, Y_pred))